#  RAG-1A - Doing RAG with MongoDB

## Basic Setup

In [1]:
## Check if GPU is enabled
import os
import torch

## To disable GPU and experiment, uncomment the following line
## Normally, you would want to use GPU, if one is available.
# os.environ["CUDA_VISIBLE_DEVICES"]=""

print ("using CUDA/GPU: ", torch.cuda.is_available())

for i in range(torch.cuda.device_count()):
   print("device ", i , torch.cuda.get_device_properties(i).name)

using CUDA/GPU:  True
device  0 NVIDIA GeForce RTX 2070


In [2]:
## Setup logging.  To see more loging set the level to DEBUG

import sys
import logging

# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Step-1: Load Settings

In [3]:
import os,sys
## Load Settings from .env file
from dotenv import find_dotenv, dotenv_values

# _ = load_dotenv(find_dotenv()) # read local .env file
config = dotenv_values(find_dotenv())

# debug
# print (config)

ATLAS_URI = config.get('ATLAS_URI')

if not ATLAS_URI:
    raise Exception ("'ATLAS_URI' is not set.  Please set it above to continue...")

## Only need this if we are using OpenAI for Embeddings
# OPENAI_API_KEY = config.get("OPENAI_API_KEY")
# if not OPENAI_API_KEY:
#     raise Exception ("'OPENAI_API_KEY' is not set.  Please set it above to continue...")

In [4]:
DB_NAME = 'rag1'
COLLECTION_NAME = '10k'
INDEX_NAME = 'idx_embedding'

In [5]:
import os
## LlamaIndex will download embeddings models as needed.
## Set llamaindex cache dir to ./cache dir here (Default is system tmp)
## This way, we can easily see downloaded artifacts
os.environ['LLAMA_INDEX_CACHE_DIR'] = os.path.join(os.path.abspath(''), 'cache')

In [6]:
import pymongo

mongodb_client = pymongo.MongoClient(ATLAS_URI)

print ("Atlas client initialized")

Atlas client initialized


## Clear out the collection

For a fresh start!

In [7]:
database = mongodb_client[DB_NAME]
collection = database [COLLECTION_NAME]

doc_count = collection.count_documents (filter = {})
print (f"Document count before delete : {doc_count:,}")

result = collection.delete_many(filter= {})
print (f"Deleted docs : {result.deleted_count}")

Document count before delete : 754
Deleted docs : 754


## Setup Embeddings

The default embedding is OpenAI.  We can always plugin custom embeddings

### OpenAI Embeddings

This is using OpenAI embedding model
You will need an API key (defined in env variable : OPENAI_API_KEY)

In [8]:
# from llama_index import  OpenAIEmbedding
# embed_model = OpenAIEmbedding()

### Using Custom Embeddings

Here are a select models for comparison.  Taken from leaderboard : https://huggingface.co/spaces/mteb/leaderboard

| model name                              | overall score | model size | model params | embedding length | License  | url                                                            |
|-----------------------------------------|---------------|------------|--------------|------------------|----------|----------------------------------------------------------------|
| BAAI/bge-large-en-v1.5                  | 64.x          | 1.34 GB    | 335 M        | 1024             | MIT      | https://huggingface.co/BAAI/bge-large-en-v1.5                  |
| BAAI/bge-small-en-v1.5                  | 62.x          | 133 MB     | 33.5 M       | 384              | MIT      | https://huggingface.co/BAAI/bge-small-en-v1.5                  |
| sentence-transformers/all-mpnet-base-v2 | 57.8          | 438 MB     |              | 768              | Apache 2 | https://huggingface.co/sentence-transformers/all-mpnet-base-v2 |
| sentence-transformers/all-MiniLM-L12-v2 | 56.x          | 134 MB     |              | 384              | Apache 2 | https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2 |
| sentence-transformers/all-MiniLM-L6-v2  | 56.x          | 91 MB      |              | 384              | Apache 2 | https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2  |

In [9]:
from llama_index.embeddings import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

/home/sujee/anaconda3/envs/atlas-2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
## setup embed model

# The LLM used to generate natural language responses to queries.
# If not provided, defaults to gpt-3.5-turbo from OpenAI
# If your OpenAI key is not set, defaults to llama2-chat-13B from Llama.cpp
# We don't need an LLM just yet, so setting it to None

from llama_index import  ServiceContext


service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=None)

LLM is explicitly disabled. Using MockLLM.


## Connect Illama-Index and MongoDB Atlas

Let's define MongoDB Atlas as our vector storage. This is critical to stored indexed data and then query

In [11]:
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from llama_index.storage.storage_context import StorageContext


vector_store = MongoDBAtlasVectorSearch(mongodb_client = mongodb_client,
                                 db_name = DB_NAME, collection_name = COLLECTION_NAME,
                                 index_name  = 'idx_embedding',
                                 ## the following columns are set to default values
                                 # embedding_key = 'embedding', text_key = 'text', metadata_= 'metadata',
                                 )
storage_context = StorageContext.from_defaults(vector_store=vector_store)

## Read Documents

Ilmaa-index has very handy `SimpleDirectoryReader` that can read single files / multiple files / or entire directory content

In [12]:
%%time

from llama_index.readers.file.base import SimpleDirectoryReader

data_dir = './data/10k/'

## This reads one doc
# docs = SimpleDirectoryReader(
#     input_files=["./data/10k/uber_2021.pdf"]
# ).load_data()

## here we read entire directory content
docs = SimpleDirectoryReader(
        input_dir=data_dir
).load_data()

print (f"Loaded {len(docs)} chunks from '{data_dir}'")

Loaded 545 chunks from './data/10k/'
CPU times: user 10.1 s, sys: 13 ms, total: 10.1 s
Wall time: 10.1 s


## Index the docs and Store 

When we execute the code below, the following will happen

- documents are indexed
- embeddings are created for text
- the document (text, embeddings) are stored in our Vector Storage (MongoDB Atlas in this case)

In [13]:
%%time

from llama_index.indices.vector_store.base import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    docs, storage_context=storage_context,
    service_context=service_context,
)

CPU times: user 9.86 s, sys: 341 ms, total: 10.2 s
Wall time: 15.7 s


## Setup Index

Before we do search, we need to define an embedding index

You can look at steps here [setup-atlas-index.md](setup-atlas-index.md)

Here are the details:

index_name = 'idx_embedding'

index defitintion json

```json
{
  "fields": [
    {
      "type": "vector",
      "path": "embedding",
      "numDimensions": 384,
      "similarity": "dotProduct"
    }
  ]
}
```

The similarity function can be  one of 
- "euclidean"
- "cosine"
- "dotProduct"

## We are done 

Now the data is populated and ready to be queried.

Let's go to the next lab: query